In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [361]:
class RandomGrid:    # Environment
    def __init__(self, width, height, start):
        self.width = width
        self.height = height
        self.i = start[0]
        self.j = start[1]
    
    def set(self, rewards, actions):
        # rewards should be a dictionary of: (i, j): r, or: (row, col): reward
        # actions should be a dictionary of: (i, j): A or (row, col): list of possible actions
        self.rewards = rewards
        self.actions = actions
        
    def set_state(self, s):
        # The state s is the location of the player in the grid: s = (i, j)
        self.i = s[0]
        self.j = s[1]
        
    def current_state(self):
        return (self.i, self.j)
    
    def is_terminal(self, s):
        # A terminal state won't be in the actions dictionary (since it won't have any associated action)
        return s not in self.actions
    
    def move(self, action):
        # check if legal move first
        # Possible actions: U/D/L/R
        if action in self.actions[self.current_state()]:
            r = np.random.random()
            remaining_actions = list(self.actions[self.current_state()])
            # if r > 0.5, use the given action, else chose a different one
            if r < 0.5:
                p = 0.5 / 3    # prob of any other action occuring
                remaining_actions.remove(action)   # if r < 0.5, then the given action will not be performed
                n = len(remaining_actions)
                action_changed = False    # if it stays false, then the action we are attempting to choose is illegal, so do nothing
                for i in range(n):
                    if i * p <= r < (i + 1) * p:
                        action = remaining_actions[i]
                        action_changed = True
                        break
                if not action_changed:
                    action = ''
            
            if action == 'U':
                self.i -= 1
            elif action == 'D':
                self.i += 1
            elif action == 'R':
                self.j += 1
            elif action == 'L':
                self.j -= 1
                
    def move_without_randomness(self, action):
        # check if legal move first
        # Possible actions: U/D/L/R
        if action in self.actions[self.current_state()]:
            if action == 'U':
                self.i -= 1
            elif action == 'D':
                self.i += 1
            elif action == 'R':
                self.j += 1
            elif action == 'L':
                self.j -= 1
        # return reward (if any)
        return self.rewards.get(self.current_state(), 0)
               
        # return action selected and reward (if any)
        return action, self.rewards.get(self.current_state(), 0)
    
    def undo_move(self, action):
        if action == 'U':
            self.i += 1
        elif action == 'D':
            self.i -= 1
        elif action == 'R':
            self.j -= 1
        elif action == 'L':
            self.j += 1
        # raise an exception if we arrive somewhere we shouldn't be
        # should never happen
        assert(self.current_state() in self.all_states())
        
    def game_over(self):
        # The game is over if we are in a state where no action is possible
        return self.current_state() not in self.actions
    
    def all_states(self):
        # Cast to a set to avoid repetition in states
        return set(list(self.rewards.keys()) + list(self.actions.keys()))
    
    def draw_grid(self):
        states = self.all_states()
        for i in range(self.height):
            for j in range(self.width):
                s = (i, j)
                symbol = ''
                if s in states:
                    if self.current_state() == s:
                        symbol = 's'
                    else:
                        symbol = '.'
                else:
                    symbol = 'x'
                print(symbol, end = '')
                if j != self.width - 1:
                    print('   ', end = '')
            print('\n')

In [321]:
def standard_random_grid():
    # Define a grid that describes the reward for arriving at each state
    #     and possible actions at each state
    # The grid looks like this
    # x means you can't go there
    # s means start position
    # number means reward at that state
    # .   .   .   1
    # .   x   .  -1
    # s   .   .   .
    g = RandomGrid(4, 3, (2, 0))
    rewards = {(0, 3): 1, (1, 3): -1}
    actions = {
        (0,0): ('D', 'R'),
        (0, 1): ('L', 'R'),
        (0, 2): ('D', 'L', 'R'),
        (1, 0): ('U', 'D'),
        (1, 2): ('U', 'D', 'R'),
        (2, 0): ('U', 'R'),
        (2, 1): ('L', 'R'),
        (2, 2): ('U', 'L', 'R'),
        (2, 3): ('U', 'L')
    }
    g.set(rewards, actions)
    return g

In [322]:
SMALL_ENOUGH = 10e-8    # threshold for convergence

In [323]:
# V is the value function dictionary, and g is the grid (environment)
def print_values(V, g):
    for i in range(g.height):
        print('-------------------------')
        print('|', end = "")
        for j in range(g.width):
            v = V.get((i, j), 0)
            if v >= 0:
                print(" %.2f|" % v, end = "")
            else:
                print("%.2f|" % v, end = "")    # negative sign takes up an extra space
        print()
    print('-------------------------')

In [324]:
# P is the policy dictionary (Mapping each state to the action to take)
def print_policy(P, g):
    for i in range(g.height):
        print('-------------------------')
        print('|', end = '')
        for j in range(g.width):
            a = P.get((i, j), ' ')
            print('  %s  |' % a, end = '')
        print()
    print('-------------------------')

In [325]:
def negative_random_grid():
    # Define a grid that describes the reward for arriving at each state
    #     and possible actions at each state
    # The grid looks like this
    # x means you can't go there
    # s means start position
    # number means reward at that state
    # .   .   .   1
    # .   x   .  -1
    # s   .   .   .
    g = RandomGrid(4, 3, (2, 0))
    actions = {
        (0,0): ('D', 'R'),
        (0, 1): ('L', 'R'),
        (0, 2): ('D', 'L', 'R'),
        (1, 0): ('U', 'D'),
        (1, 2): ('U', 'D', 'R'),
        (2, 0): ('U', 'R'),
        (2, 1): ('L', 'R'),
        (2, 2): ('U', 'L', 'R'),
        (2, 3): ('U', 'L')
    }
    rewards = {(0, 3): 1, (1, 3): -1}
    
    # Penalise the player for each step, to see if he can finish with the minimum step
    for s in actions.keys():
        rewards[s] = -0.1
    g.set(rewards, actions)
    return g

In [326]:
g = negative_random_grid()

In [327]:
def get_state_transition_prob(s, a, g):
    if a not in g.actions[s]:
        return {}
    # Returns the possible future states and the probability to get to each state, given that we are at state s and we perform action a
    prob_dict = {}
    i, j = s[0], s[1]
    # Find the next state if we perform action a and action a actually happens
    if a == 'U':
        i -= 1
    elif a == 'D':
        i += 1
    elif a == 'L':
        j -= 1
    elif a == 'R':
        j += 1
    # The chances of action a actually happening are 0.5
    prob_dict[(i, j)] = 0.5
    # Find the next states if we perform action a but other actions happen
    for action in g.actions[s]:
        i, j = s[0], s[1]
        if action != a:
            if action == 'U':
                i -= 1
            elif action == 'D':
                i += 1
            elif action == 'L':
                j -= 1
            elif action == 'R':
                j += 1
            prob_dict[(i, j)] = 0.5 / 3
    return prob_dict
        

In [328]:
# Get the value function corresponding to the given policy
### Iterative Policy Evaluation
### Here the policy is given (deterministic)
### The actions are deterministic
### The state transitions are probabilistic
def get_value_function(policy, g):
    states = g.all_states()
    # initialize V(s) = 0
    V = {}
    for s in states:
        V[s] = 0
    gamma = 0.9
    while True:
        delta = 0
        for s in states:
            if not g.is_terminal(s):
                g.set_state(s)
                old_v = V[s]
                action = policy[s]
                states_prob = get_state_transition_prob(s, action, g)
                V[s] = 0
                for s_prime in states_prob.keys():
                    p = states_prob[s_prime]
                    if s_prime in g.rewards.keys():
                        r = g.rewards[s_prime]
                    else:
                        r = 0
                    V[s] += p * (r + gamma * V[s_prime])
                delta = max(delta, abs(V[s] - old_v))
        if delta < SMALL_ENOUGH:
            break
    return V
        

In [329]:
g.draw_grid()

.   .   .   .

.   x   .   .

s   .   .   .



In [330]:
policy = {
    (0,0):'R',
    (0, 1): 'R',
    (0, 2):'R',
    (1, 0):'U',
    (1, 2):'R',
    (2, 0):'U',
    (2, 1):'R',
    (2, 2):'R',
    (2, 3):'U'
}

In [295]:
V = get_value_function(policy, g)

In [296]:
print_values(V, g)

-------------------------
|-0.03| 0.11| 0.40| 0.00|
-------------------------
|-0.11| 0.00|-0.54| 0.00|
-------------------------
|-0.16|-0.30|-0.48|-0.59|
-------------------------


In [297]:
print_policy(policy, g)

-------------------------
|  R  |  R  |  R  |     |
-------------------------
|  U  |     |  R  |     |
-------------------------
|  U  |  R  |  R  |  U  |
-------------------------


In [298]:
get_state_transition_prob((0, 1), 'L', g)

{(0, 0): 0.5, (0, 2): 0.16666666666666666}

In [299]:
### Policy Iteration Algorithm
### Finds the optimal policy, starting with a random policy and ameliorating it
def policy_iteration(g):
    policy = {}
    
    # Randomly initialize policy
    # Since the policy is a state -> action mapping, randomly initializing the policy is randomly choosing an action for each non terminal state
    for s in g.actions.keys():
        policy[s] = np.random.choice(g.actions[s])
        
        
    policy_changed = True
        
    # Keep updating the policy (by finding better actions for each state) until the policy doesn't change anymore
    while policy_changed:
        V = get_value_function(policy, g)
        policy_changed = False
        gamma = 0.9
        states = g.all_states()
        for s in states:
            if not g.is_terminal(s):
                g.set_state(s)
                old_a = policy[s]    # Old action determined by the old policy
                # Find the best action (action with highest value)
                max_a = old_a
                max_val = float('-inf')
                for a in g.actions[s]:
                    states_prob = get_state_transition_prob(s, a, g)
                    val = 0
                    # Get all next possible states and their probabilities to calculate value
                    for s_prime in states_prob.keys():
                        p = states_prob[s_prime]
                        if s_prime in g.rewards.keys():
                            r = g.rewards[s_prime]
                        else:
                            r = 0
                        val += p * (r + gamma * V[s_prime])
                    # Update max value and corresponding action
                    if val > max_val:
                        max_a = a
                        max_val = val
                policy[s] = max_a
                # if the new action is different from the one deyermined by the old policy, then the policy has changed
                if policy[s] != old_a:
                    policy_changed = True
    return policy, V

In [300]:
policy, V = policy_iteration(g)

In [301]:
print_policy(policy, g)

-------------------------
|  R  |  R  |  R  |     |
-------------------------
|  U  |     |  U  |     |
-------------------------
|  U  |  L  |  U  |  L  |
-------------------------


In [302]:
print_values(V, g)

-------------------------
|-0.01| 0.15| 0.48| 0.00|
-------------------------
|-0.09| 0.00|-0.04| 0.00|
-------------------------
|-0.13|-0.15|-0.17|-0.29|
-------------------------


In [399]:
def get_policy(V, g):
    # given the value function, this method finds the value function greedily
    gamma = 0.9
    policy = {}
    states = g.all_states()
    for s in states:
        max_val = float('-inf')
        arg_max = 0
        if not g.is_terminal(s):
            #print('state:', s)
            #print(' possible actions:', g.actions[s])
            g.set_state(s)
            # for each action, perform it, find the value of the next state. 
            # Save the one that provided the max value
            for a in g.actions[s]:
                states_prob = get_state_transition_prob(s, a, g)
                r = g.move_without_randomness(a)
                p = states_prob[g.current_state()]
                val = p * (r + gamma * V[g.current_state()])
                #print('  ', a, val)
                if val > max_val:
                    max_val = val
                    argmax = a
                g.undo_move(a)
            #print(' action taken:', argmax)
            policy[s] = argmax
    return policy

In [400]:
def value_iteration(g):
    gamma = 0.9
    states = g.all_states()
    V = {}
    for s in states:
        V[s] = 0
    
    while True:
        delta = 0
        for s in states:
            if not g.is_terminal(s):
                old_v = V[s]
                max_a_val = float('-inf')
                for a in g.actions[s]:
                    val = 0
                    # get the probabilities of each action to happen at that state
                    states_prob = get_state_transition_prob(s, a, g)
                    for s_prime in states_prob.keys():
                        p = states_prob[s_prime]
                        if s_prime in g.rewards.keys():
                            r = g.rewards[s_prime]
                        else:
                            r = 0
                        val += p * (r + gamma * V[s_prime])
                    # Find the action that yields the maximum value
                    if val > max_a_val:
                        max_a_val = val
                # Set the value of that state to be that maximum value (So that the value function is optimal already)
                V[s] = max_a_val
                delta = max(delta, abs(V[s] - old_v))
        if delta < SMALL_ENOUGH:
            break
    
    # Once we're done with the value function, find the corresponding policy greedily
    policy = get_policy(V, g)
        
    return policy, V

In [404]:
g = standard_random_grid()
policy, V = value_iteration(g)

In [405]:
print_policy(policy, g)

-------------------------
|  R  |  R  |  R  |     |
-------------------------
|  U  |     |  U  |     |
-------------------------
|  U  |  L  |  U  |  L  |
-------------------------


In [406]:
print_values(V, g)

-------------------------
| 0.13| 0.27| 0.55| 0.00|
-------------------------
| 0.06| 0.00| 0.08| 0.00|
-------------------------
| 0.03| 0.02| 0.02|-0.16|
-------------------------
